# Severe Weather Capstone - Data Collection & Wrangling

Greg Welliver   

In [1]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm

from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats
import re
from glob import glob, iglob
from datetime import datetime
import string


In [2]:
#NotebookApp.iopub_data_rate_limit=1.0e10
#https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image

file location for downloads: 
    https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/

## Data Collection

- storm files were collected from the Iowa Environmental Mesonet: https://mesonet.agron.iastate.edu/nws/

### working code, make markdown for now
### All annual storm data files are saved on my local machine.  This code gathers all of the files and combines them into one file.
filenames = glob('../Data/*.csv')
print("There is a total of {} files.".format(len(filenames)))

target_path = '../Data/all_storm_data.csv'

try:
    # Read in Summary File is exists
    all_storm_data = pd.read_csv(target_path)
except:
    # Read in all Subfiles
    storm_data = [pd.read_csv(filepath) for filepath in filenames]
    all_storm_data = pd.concat(storm_data)
    
    # Create Summary File for faster processing
    hot100_all.to_csv(target_path,index=False)

print("The total number of observations is {}.".format(len(all_storm_data)))
all_storm_data.head()

In [3]:
# load data
#df = pd.read_csv("../Data/StormEvents_details-ftp_v1.0_d2001_c20220425.csv")
#df = pd.read_parquet("../Data/all_storm_data.pqt")
#df = pd.read_csv("../Data/all_storm_data4.csv")

In [4]:
#df.head()

In [5]:
#df.shape

In [6]:
#df.isna().sum()

In [7]:
#df.info()

for row in df["STATE_FIPS"][:10]:
    res = row.split(".", 1)[0]
    print(res)

# WORKING, MARKDOWN UNTIL FINAL
# drop unnecessary columns
df.drop(['CATEGORY', 'DATA_SOURCE', 'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'CZ_TIMEZONE', 'WFO', 'CZ_TYPE', 'DAMAGE_CROPS', 'CZ_NAME', 'SOURCE', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE_FIPS', 'CZ_FIPS', 'END_DATE_TIME'], axis=1, inplace=True)

# WORKING, MARKDOWN UNTIL FINAL
df['STATE_FIPS'] = df['STATE_FIPS'].astype(object)

# WORKING, MARKDOWN UNTIL FINAL
# Columns to replace nulls with NA:
cols_na = ['EVENT_NARRATIVE', 'EPISODE_NARRATIVE', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON', 'TOR_F_SCALE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'STATE', 'STATE_FIPS']

for x in cols_na:
    print(df[x].isna().sum())

# WORKING, MARKDOWN UNTIL FINAL
for x in cols_na:
    df[x] = df[x].fillna('NA')

for x in cols_na:
    print(df[x].isna().sum())

# WORKING, MARKDOWN UNTIL FINAL
# Columns to replace nulls with 0:

cols_0 = ['MAGNITUDE', 'TOR_LENGTH', 'TOR_WIDTH', 'DAMAGE_PROPERTY', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT']


for x in cols_0:
    print(df[x].isna().sum())

# WORKING, MARKDOWN UNTIL FINAL
for x in cols_0:
    df[x] = df[x].fillna(0)

for x in cols_0:
    print(df[x].isna().sum())

In [8]:
#df.isna().sum()

In [9]:
#df.dtypes

# WORKING, MARKDOWN UNTIL FINAL
# remove NA values from state FIPS
df = df[df['STATE_FIPS'] != "NA"].reset_index()

# convert STATE_FIPS to INT so can use it for lookup later
for x in df['STATE_FIPS']:
    x = int(x)

df['STATE_FIPS'] = df['STATE_FIPS'].astype(int)


df['STATE_FIPS'] = df['STATE_FIPS'].astype(object)

df['STATE_FIPS']

df['CZ_FIPS'] = df['CZ_FIPS'].astype(str)

df['CZ_FIPS']

df['CZ_FIPS'][:5]

In [10]:
#df[249750:249760]

df['CZ_FIPS'][249750:249760]

# WORKING, MARKDOWN UNTIL FINAL
# add "0" or "00" to CZ FIPS so that it can be used to match later
for i in (range(len(df['CZ_FIPS']))):
    if len(df['CZ_FIPS'][i]) == 2:
#        df['CZ_FIPS'][i] = df['CZ_FIPS'][i].astype(str)
        df['CZ_FIPS'][i] = "0" + df['CZ_FIPS'][i]
#        print(df['CZ_FIPS'][i])
    elif len(df['CZ_FIPS'][i]) == 1:
        df['CZ_FIPS'][i] = "00" + df['CZ_FIPS'][i]
#        print(df['CZ_FIPS'][i])
#     else:
#         row

df['STATE_FIPS'] = df['STATE_FIPS'].astype(str)

# WORKING, MARKDOWN UNTIL FINAL
# add "0" to state FIPS so that it can be used to match later
for i in (range(len(df['STATE_FIPS']))):
    if len(df['STATE_FIPS'][i]) == 1:
#        df['CZ_FIPS'][i] = df['CZ_FIPS'][i].astype(str)
        df['STATE_FIPS'][i] = "0" + df['STATE_FIPS'][i]
#        print(df['CZ_FIPS'][i])

#df["CZ_FIPS"] = df.apply(lambda x: "0" + x if len(x) == 2)

# WORKING, MARKDOWN UNTIL FINAL
# concatenate STATE FIPS and CZ FIPS into one column so that it can be used to match
df['ST_CT_FIPS'] = df['STATE_FIPS'].astype(str) + df['CZ_FIPS'].astype(str)

# WORKING, MARKDOWN UNTIL FINAL
# remove all of the K's, M's, and B's in the DAMAGE_PROPERTY column and multiply them by appropriate values
d = {r"(\d)K$": r"\1*1000", r"M$": r"*1000000", r"B$": r"*1000000000", r"^K$": r"1000"}

#r stands for raw string
#dollar is end of the line

# for every key and value, run this code
for k,v in d.items():
     df["DAMAGE_PROPERTY"] = df["DAMAGE_PROPERTY"].str.replace(k, v, regex=True).fillna("0.0")
#df["DAMAGE_PROPERTY"].apply(eval)
df["DAMAGE_PROPERTY"] = df["DAMAGE_PROPERTY"].apply(eval)

# WORKING, MARKDOWN UNTIL FINAL
# convert date strings to datetimes
df['BEGIN_DATE_TIME'] =  pd.to_datetime(df['BEGIN_DATE_TIME'])
df['END_DATE_TIME'] =  pd.to_datetime(df['END_DATE_TIME'])

# WORKING, MARKDOWN UNTIL FINAL
# calculate duration of storm
df['DURATION'] = df['END_DATE_TIME'] - df['BEGIN_DATE_TIME']

# convert storm duration to minutes
for i in (range(len(df['DURATION']))):
    df['DURATION'][i] = df['DURATION'][i].total_seconds() / 60

# WORKING, MARKDOWN UNTIL FINAL
# code to calculate coverage area of the storm

# calculate  beginning and end latitude difference
df['LAT_DIFF'] = (df['END_LAT'] - df['BEGIN_LAT']).abs()

# calculate  beginning and end longitude difference
df['LON_DIFF'] = (df['END_LON'] - df['END_LON']).abs()

# combine two columns to calculate total size of storm
df['STORM_AREA'] = df['LON_DIFF'] + df['LAT_DIFF']

# since we don't need the difference columns anymore, drop those. also END LAT and LON columsn, since don't need those either
df.drop(['LAT_DIFF', 'LON_DIFF', 'END_LON', 'END_LAT'], axis=1, inplace=True)

## Part 2 start here

In [11]:
#load the data
# df = pd.read_csv("../Data/all_storm_data4.csv", index_col=[0])
# df.drop(['index'], axis=1, inplace=True)
df = pd.read_parquet("../Data/all_storm_data7.pqt")

In [12]:
df.head()

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA
0,202202,2118,NEVADA,2022,February,High Wind,2022-02-20 21:18:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Strong winds increased ahead of an approaching...,"Station (UP994) 3.1 SE West Wendover, Elevatio...",32033,60.0,NaN
1,202202,800,NEVADA,2022,February,Heavy Snow,2022-02-21 08:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,32037,1560.0,NaN
2,202202,200,NEVADA,2022,February,Heavy Snow,2022-02-22 02:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Fifteen inches fell at station (TJMN2) Toe Jam...,32031,420.0,NaN
3,202202,1609,ATLANTIC SOUTH,2022,February,Waterspout,2022-02-18 16:09:00,0,0,0,...,None,0.0,0.0,30.050,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,87452,0.0,0.0000
4,202202,0,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-02 00:00:00,0,0,0,...,None,0.0,0.0,-14.333,-170.7157,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",97002,1440.0,0.0063


In [13]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
BEGIN_YEARMONTH,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202
BEGIN_TIME,2118,800,200,1609,0,500,100,100,200,400,400,300,500,2026,900,900,900,900,900,900
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA,AMERICAN SAMOA,KANSAS,KANSAS,MASSACHUSETTS,MASSACHUSETTS,MASSACHUSETTS,RHODE ISLAND,FLORIDA,MONTANA,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK
YEAR,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain,Heavy Rain,Winter Storm,Winter Storm,Heavy Snow,Heavy Snow,Heavy Snow,Heavy Snow,Dense Fog,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00,2022-02-12 05:00:00,2022-02-01 01:00:00,2022-02-01 01:00:00,2022-02-13 02:00:00,2022-02-13 04:00:00,2022-02-13 04:00:00,2022-02-13 03:00:00,2022-02-25 05:00:00,2022-02-01 20:26:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00
INJURIES_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df['ST_CT_FIPS'] = df['ST_CT_FIPS'].astype(str)
df['ST_CT_FIPS'] = df['ST_CT_FIPS'].str.zfill(5)
df.ST_CT_FIPS

0          32033
1          32037
2          32031
3          87452
4          97002
           ...  
1555642    99001
1555643    05109
1555644    99001
1555645    99004
1555646    99005
Name: ST_CT_FIPS, Length: 1555647, dtype: object

In [15]:
df.head()

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA
0,202202,2118,NEVADA,2022,February,High Wind,2022-02-20 21:18:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Strong winds increased ahead of an approaching...,"Station (UP994) 3.1 SE West Wendover, Elevatio...",32033,60.0,NaN
1,202202,800,NEVADA,2022,February,Heavy Snow,2022-02-21 08:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,32037,1560.0,NaN
2,202202,200,NEVADA,2022,February,Heavy Snow,2022-02-22 02:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Fifteen inches fell at station (TJMN2) Toe Jam...,32031,420.0,NaN
3,202202,1609,ATLANTIC SOUTH,2022,February,Waterspout,2022-02-18 16:09:00,0,0,0,...,None,0.0,0.0,30.050,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,87452,0.0,0.0000
4,202202,0,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-02 00:00:00,0,0,0,...,None,0.0,0.0,-14.333,-170.7157,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",97002,1440.0,0.0063


In [16]:
df.head().T

,0,1,2,3,4
BEGIN_YEARMONTH,202202,202202,202202,202202,202202
BEGIN_TIME,2118,800,200,1609,0
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA
YEAR,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00
INJURIES_DIRECT,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0


In [17]:
df.ST_CT_FIPS.sort_values(ascending=False)

402724     99153
1004930    99153
1217216    99153
1117745    99153
894782     99153
           ...  
476623     01001
214390     01001
84059      01001
493378     01001
393601     01001
Name: ST_CT_FIPS, Length: 1555647, dtype: object

## Combine Population Density, Home Price data


In [ ]:
#load the data
#PopDen = pd.read_csv("../Data/Average_Household_Size_and_Population_Density_-_County_merge.csv", index_col=[0])
PopDen = pd.read_csv("../Data/Average_Household_Size_and_Population_Density_-_County_merge.csv")
#HomePrice = pd.read_csv("../Data/HPI_AT_BDL_county_merge.csv", index_col=[0])
#HomePrice = pd.read_csv("../Data/HPI_AT_BDL_county_merge.csv")
HomePrice = pd.read_excel("../Data/HPI_AT_BDL_county_merge.xlsx")

In [ ]:
PopDen = PopDen[PopDen['FIPS_CODE'].notnull()]
PopDen = PopDen.reset_index(drop=True)

In [ ]:
# code to fix FIPS_CODE column in PopDen

# convert to int to get rid of decimals
PopDen['FIPS_CODE'] = PopDen['FIPS_CODE'].astype(int)

# pad additional zeroes
PopDen['FIPS_CODE'] = PopDen['FIPS_CODE'].astype(str)
PopDen['FIPS_CODE'] = PopDen['FIPS_CODE'].str.zfill(5)

# code to fix FIPS CODE column in HomePrice

# pad additional zeroes
HomePrice['FIPS code'] = HomePrice['FIPS code'].astype(str)
HomePrice['FIPS code'] = HomePrice['FIPS code'].str.zfill(5)

In [ ]:
PopDen.head(20).T

In [ ]:
HomePrice.head(20).T

In [ ]:
df[df['ST_CT_FIPS'] == "01001"]

In [ ]:
# working
# merge the population density data to the main dataframe
df = df.merge(PopDen['B01001_calc_PopDensity'], how = 'left',
                left_on = 'ST_CT_FIPS', right_on = PopDen['FIPS_CODE'])
#TopCountries.index = TopCountries.index + 1

# using pd.concat
# merge the population density data to the main dataframe
df = pd.concat([df, PopDen[['B01001_calc_PopDensity', 'Population']]],
                  keys = ['ST_CT_FIPS', 'FIPS_CODE'])
#TopCountries.index = TopCountries.index + 1

In [ ]:
df.head(30).T

In [ ]:
### FIXING BELOW

In [ ]:
# merge the population data to the main dataframe
df = df.merge(PopDen['Population'], how = 'left',
                left_on = 'ST_CT_FIPS', right_on = PopDen['FIPS_CODE'])
#TopCountries.index = TopCountries.index + 1

In [ ]:
df[df['ST_CT_FIPS'] == "01001"]

In [ ]:
#### FIXING ABOVE

In [ ]:
# merge the home price index data to the main dataframe
df = pd.merge(df, HomePrice,  how='left', left_on=['ST_CT_FIPS','YEAR'], right_on = ['FIPS code','Year'])
df.drop(['HPI with 2000 base', 'HPI with 1990 base', 'Annual Change (%)', 'Year', 'FIPS code', 'County', 'State',], axis=1, inplace=True)


In [ ]:
df[df['ST_CT_FIPS'] == "01001"]

In [ ]:
df.info()

In [ ]:
df.head(30).T

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.head(20).T

In [ ]:
df.HPI

In [ ]:
# code to replace "." so that we can ultimately turn this into a number

new_list = []
for i in df['HPI']:
    a_string = str(i)
    #a_string = df['HPI'][i]
    new_string = a_string.translate(str.maketrans('', '', string.punctuation))
    #print(a_string)
    #df['new_HPI'][i] = new_string
    #print(df['new_HPI'][i])
    new_list.append(new_string)
#    print(df['HPI'][i])    
#     df['new_HPI'][i] = new_string
#     print(df['new_HPI'][i])
#print(new_list)

In [ ]:
df.HPI.notnull().value_counts()

In [ ]:
df.HPI.value_counts()

In [ ]:
df['newHPI'] = pd.DataFrame(new_list)
#df['newHPI'] = df['newHPI'].replace('nan', 'NaN')
#df['newHPI'] = df['newHPI'].replace('', 'NaN')

In [ ]:
df['newHPI']

In [ ]:
df[df['newHPI'] == "nan"]

In [ ]:
df.info()

In [ ]:
df.newHPI.value_counts()

In [ ]:
df.newHPI.notnull().value_counts()

In [ ]:
df.info()

In [ ]:
df[df['ST_CT_FIPS'] == "01001"]

# START HERE

In [ ]:
#df = pd.read_parquet("../Data/all_storm_data9.pqt")
#df = pd.read_parquet("../Data/all_storm_data9.pqt", index_col=[0])

In [ ]:
df.head()

In [ ]:
#fixing the merge

In [ ]:
df.head(20).T

In [ ]:
df.PopDensity.value_counts()

In [ ]:
PopDen.head(20).T

In [ ]:
df

In [ ]:
df.drop(['HPI'], axis=1, inplace=True)

In [ ]:
df = df.rename(columns={'B01001_calc_PopDensity': 'PopDensity'})

In [ ]:
df.info()

In [ ]:
df = df.rename(columns={'newHPI': 'HPI'})

In [ ]:
df['HPI'] = df['HPI'].astype(int)

#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Severe-Weather-Repo/Data/all_storm_data5.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

#### write to parquet
parquet_file = 'example_pd.parquet'

df.to_parquet(parquet_file, engine = 'pyarrow', compression = 'gzip')

logging.info('Parquet file named "%s" has been written to disk', parquet_file)

In [ ]:
#### write to parquet
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Severe-Weather-Repo/Data/all_storm_data10.pqt')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_parquet(filepath)

resources

CZ FIPS documentation: https://www.irsa.miami.edu/_assets/pdf/Documents/fips_statecounty_code.pdf

Population density: https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf_1/explore?location=2.632620%2C0.315550%2C1.00

Home price index: https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx

maybe useful: https://www.nar.realtor/research-and-statistics/housing-statistics/county-median-home-prices-and-monthly-mortgage-payment
        
land values: https://www.nass.usda.gov/Publications/Todays_Reports/reports/land0822.pdf

data that I created:
 - concatenated state and county codes for indentification
 - storm duration
 - storm area
 - county population density (pulled from other dataset)
 - land values (pulled from other dataset)

drop columns
# WORKING, MARKDOWN UNTIL FINAL
# drop unnecessary columns
df.drop(['', '', '', '',], axis=1, inplace=True)

# working
# merge the home price index data to the main dataframe
df = df.merge(HomePrice['HPI'], how = 'left',
                left_on = 'ST_CT_FIPS', right_on = HomePrice['FIPS code'])
#TopCountries.index = TopCountries.index + 1